In [1]:
pip install pandas networkx matplotlib folium osmnx


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 645.7 kB/s eta 0:00:02
   ------------------ --------------------- 0.8/1.7 MB 671.3 kB/s eta 0:00:02
   ------------------ --------------------- 0.8/1.7 MB 671.3 kB/s eta 0:00:02
   ------------------------ --------------- 1.0/1.7 MB 671.0 kB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.7 MB 671.0 kB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 706.6 kB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 706.6 kB/s eta 0:00:01
   ----------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import geopandas as gpd
import networkx as nx

# Load GeoJSON
gdf = gpd.read_file("turin.geojson")

print(gdf.head())   # check columns


             id           @id access access_1 addr:city addr:housenumber  \
0  way/22885550  way/22885550   None     None      None             None   
1  way/22885660  way/22885660   None     None      None             None   
2  way/22885664  way/22885664   None     None      None             None   
3  way/22897749  way/22897749   None     None      None             None   
4  way/23054095  way/23054095   None     None      None             None   

  addr:postcode addr:street agricultural alt_name  ... usage vehicle  \
0          None        None         None     None  ...  None    None   
1          None        None         None     None  ...  None    None   
2          None        None         None     None  ...  None    None   
3          None        None         None     None  ...  None    None   
4          None        None         None     None  ...  None    None   

  vehicle:conditional voltage website wheelchair width  wikidata  \
0                None    None    None     

In [4]:
import math
from shapely.geometry import LineString, Polygon

G = nx.Graph()

for idx, row in gdf.iterrows():
    geom = row['geometry']
    
    # Case 1: If geometry is LineString (roads usually are)
    if isinstance(geom, LineString):
        coords = list(geom.coords)
    
    # Case 2: If geometry is Polygon (sometimes OSM boundaries)
    elif isinstance(geom, Polygon):
        coords = list(geom.exterior.coords)  # take outer boundary
    
    else:
        continue  # skip if not LineString/Polygon
    
    # Add edges between consecutive coords
    for i in range(len(coords)-1):
        x1, y1 = coords[i]
        x2, y2 = coords[i+1]
        
        dist = math.sqrt((x2-x1)**2 + (y2-y1)**2)
        G.add_edge((x1, y1), (x2, y2), weight=dist)

print("Graph built with", G.number_of_nodes(), "nodes and", G.number_of_edges(), "edges")


Graph built with 89676 nodes and 100880 edges


In [5]:
# Pick first and last node
source = list(G.nodes())[0]
target = list(G.nodes())[-1]

path = nx.dijkstra_path(G, source, target, weight="weight")
path_length = nx.dijkstra_path_length(G, source, target, weight="weight")

print("Shortest path:", path)
print("Path length:", path_length)


Shortest path: [(7.6934565, 45.0654618), (7.6937925, 45.0652465), (7.6937852, 45.0652407), (7.6937306, 45.0652003), (7.6937134, 45.0651882), (7.6932177, 45.0647931), (7.6931693, 45.0647547), (7.6931076, 45.0647061), (7.6917554, 45.0636396), (7.6917147, 45.0636075), (7.6916353, 45.0636352), (7.6910475, 45.0638405), (7.6909249, 45.0638833), (7.6908896, 45.06383), (7.6908147, 45.0637045), (7.690588, 45.0635927), (7.690335, 45.0634421), (7.6902516, 45.0633854), (7.690101, 45.0632831), (7.6898061, 45.0631481), (7.6896374, 45.063113), (7.6895601, 45.063097), (7.6892653, 45.0630718), (7.6890434, 45.0630875), (7.6889598, 45.0631021), (7.6888849, 45.063117), (7.6887373, 45.0631514), (7.6886006, 45.0632315), (7.6884937, 45.0632774), (7.6883559, 45.0633277), (7.6873607, 45.0636909), (7.6871448, 45.0637506), (7.6870453, 45.0636113), (7.6869131, 45.0634305), (7.6868352, 45.0634555), (7.6858291, 45.0637967), (7.6857503, 45.0638217), (7.6856578, 45.0638389), (7.6844064, 45.0639165), (7.6843463, 45.06

In [7]:
from sklearn.metrics import accuracy_score, f1_score

min_len = min(len(true_path), len(predicted_path))
accuracy = sum(1 for i in range(min_len) if true_path[i] == predicted_path[i]) / min_len
print("Path Accuracy:", accuracy)



Path Accuracy: 0.0
